In [1]:
import numpy as np
from bias_correction import bias_corrected_mine
# MINE-only imports:
from models import *
import torch.optim as optim
import matplotlib.pyplot as plt

# Import Data

In [2]:
base = '/bigstore/GeneralStorage/Evan/Calcium_Feature_Extraction/Models/MasterLibrary/NoisyCa_Gaussian_Genes_Volume_Norm_MI_090520/Learning_Rate_Test/'
data = np.load(base+'/full_data.npy')

In [3]:
genes = data[:,:83]
calcium = data[:,83:]
'Full data:', data.shape, 'Gene data:', genes.shape, 'Calcium data:', calcium.shape

('Full data:',
 (5128, 397),
 'Gene data:',
 (5128, 83),
 'Calcium data:',
 (5128, 314))

# Bias-Corrected MINE Example

In [10]:
joint_res = bias_corrected_mine().train(genes, calcium, min_iters=1e5, max_iters=4e5, 
                                        step_size=2e4, hidden_size=600, learn_rate=3e-4) 
    
init1 = np.nanpercentile(joint_res['raw_observed_mi_trajectories'], 75)  # usually %ile initialization works best
ubmine = bias_corrected_mine(mi_init1=init1)
ubmine.mi_init2 = init1-0.0001  # default behavior is mi_init1/1.5 just to try a range of inits
ans = (ubmine.refit(joint_res['raw_observed_mi_trajectories'], dynamic_start=True)/np.log(2))

In [11]:
ans  # ans[-1] is the final result with fast convergence, falls within 2.5+-0.4

array([1.72594403, 2.07329545, 2.25154215, 2.8738426 ])

# Raw MINE Example

In [5]:
mine = Mine(input_size=83+314, hidden_size=600).cuda()
mine_net_optim_joint = optim.Adam(mine.parameters(), lr=3e-4)
joint_res = train(data,mine,mine_net_optim_joint, iter_num=int(1e5), n_genes=83)
joint_res_ma = ma(joint_res, 5000)/np.log(2)  # moving average is used in the paper to correct gradient bias

In [26]:
np.mean(joint_res_ma[-5000:])  # a little bit larger without bias correction

3.0375345